In [1]:
import os
import sys
from typing import List, Tuple, Dict, Optional, Any, Union, Callable, Iterable, Iterator, NamedTuple, Set, Sequence
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torchvision.models import resnet50
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import ray
import wids
import dataclasses
import time
from collections import deque

def enumerate_report(seq, delta, growth=1.0):
    last = 0
    count = 0
    for count, item in enumerate(seq):
        now = time.time()
        if now - last > delta:
            last = now
            yield count, item, True
        else:
            yield count, item, False
        delta *= growth


/home/tmb/proj/webdataset/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-12 03:18:15,550	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
bucket = "https://storage.googleapis.com/webdataset/fake-imagenet"
trainset_url = bucket+"/imagenet-train-{000000..001281}.tar"
valset_url = bucket+"/imagenet-val-{000000..000049}.tar"

if 'google.colab' in sys.modules:
    cache_dir = None
    print("running on colab, streaming data directly from storage")
else:
    cache_dir = "./_cache"
    print(f"not running in colab, caching data locally in {cache_dir}")

not running in colab, caching data locally in ./_cache


In [3]:
# This is a typical PyTorch dataset, except that we read from the cloud.

def make_dataset_train():
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    def make_sample(sample):
        image = sample[".jpg"]
        label = sample[".cls"]
        return transform_train(image), label

    trainset = wids.ShardListDataset("gs://webdataset/fake-imagenet/imagenet-train.json", cache_dir="./_cache", keep=True)
    trainset = trainset.add_transform(make_sample)

    return trainset

# To keep locality of reference in the dataloader, we use a special sampler
# for distributed training, DistributedChunkedSampler.

def make_dataloader_train():
    dataset = make_dataset_train()
    sampler = wids.DistributedChunkedSampler(dataset, chunksize=1000, shuffle=True)
    dataloader = DataLoader(dataset, batch_size=32, sampler=sampler, num_workers=4)
    return dataloader


def make_dataloader(split="train"):
    """Make a dataloader for training or validation."""
    if split == "train":
        return make_dataloader_train()
    elif split == "val":
        return make_dataloader_val()
    else:
        raise ValueError(f"unknown split {split}")

# Try it out.
sample = next(iter(make_dataloader()))
print(sample[0].shape, sample[1].shape)

gs://webdataset/fake-imagenet/imagenet-train.json base: gs://webdataset/fake-imagenet name: imagenet-train nfiles: 1282 nbytes: 31242280960 samples: 128200 cache: ./_cache
/home/tmb/proj/webdataset/wids/wids.py:710: UserWarning: DistributedChunkedSampler is called without distributed initialized; assuming single process
  warnings.warn(


torch.Size([32, 3, 224, 224]) torch.Size([32])


In [4]:
# For convenience, we collect all the configuration parameters into
# a dataclass.

@dataclasses.dataclass
class Config:
    rank: Optional[int] = None
    epochs: int = 1
    maxsteps: int = int(1e18)
    lr: float = 0.001
    momentum: float = 0.9
    world_size: int = 2
    backend: str = "nccl"
    master_addr: str = "localhost"
    master_port: str = "12355"
    report_s: float = 15.0
    report_growth: float = 1.1

Config()

Config(rank=None, epochs=1, maxsteps=1000000000000000000, lr=0.001, momentum=0.9, world_size=2, backend='nccl', master_addr='localhost', master_port='12355', report_s=15.0, report_growth=1.1)

In [5]:
# A typical PyTorch training function.

def train(config):
    # Define the model, loss function, and optimizer
    model = resnet50(pretrained=False).cuda()
    if config.rank is not None:
        model = DistributedDataParallel(model)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=config.lr)

    # Data loading code
    trainloader = make_dataloader(split='train')

    losses, accuracies, steps = deque(maxlen=100), deque(maxlen=100), 0

    # Training loop
    for epoch in range(config.epochs):
        for i, data, verbose in enumerate_report(trainloader, config.report_s):
            inputs, labels = data[0].cuda(), data[1].cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # just bookkeping and progress report
            steps += len(labels)
            accuracy = (outputs.argmax(1) == labels).float().mean()  # calculate accuracy
            losses.append(loss.item())
            accuracies.append(accuracy.item())
            if verbose and len(losses) > 0:
                avgloss = sum(losses)/len(losses)
                avgaccuracy = sum(accuracies)/len(accuracies)
                print(f"rank {config.rank} epoch {epoch:5d}/{i:9d} loss {avgloss:8.3f} acc {avgaccuracy:8.3f} {steps:9d}", file=sys.stderr)
            if steps > config.maxsteps:
                print("finished training (maxsteps)", steps, config.maxsteps, file=sys.stderr)
                return

    print("finished Training", steps)

In [6]:
# A quick smoke test.

config = Config()
config.epochs = 1
config.maxsteps = 1000
train(config)

/home/tmb/proj/webdataset/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tmb/proj/webdataset/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
gs://webdataset/fake-imagenet/imagenet-train.json base: gs://webdataset/fake-imagenet name: imagenet-train nfiles: 1282 nbytes: 31242280960 samples: 128200 cache: ./_cache
rank None epoch     0/        0 loss    7.180 acc    0.000        32
finished training (maxsteps) 1024 1000


In [12]:
# The distributed training function to be used with Ray.
# Since this is started via Ray remote, we set up the distributed
# training environment here.

def train_in_ray(rank, config):
    # Set up distributed PyTorch.
    if rank is not None:
        config.rank = rank
        os.environ['MASTER_ADDR'] = config.master_addr
        os.environ['MASTER_PORT'] = config.master_port
        print(f"rank {rank} initializing process group", file=sys.stderr)
        dist.init_process_group(backend=config.backend, rank=rank, world_size=config.world_size)
        print(f"rank {rank} done initializing process group", file=sys.stderr)
    train(config)

In [13]:
# Initialize the Ray cluster if it hasn't been initialized yet.

if not ray.is_initialized():
    ray.init()
print("#gpus available in the cluster", ray.available_resources()['GPU'])
print(ray.nodes())

#gpus available in the cluster 2.0
[{'NodeID': '1548755e0d56fbbaea367df3cb590eeb6af479d17a2f1c9b48a2e8a4', 'Alive': True, 'NodeManagerAddress': '10.20.13.236', 'NodeManagerHostname': 'bragi', 'NodeManagerPort': 42255, 'ObjectManagerPort': 38851, 'ObjectStoreSocketName': '/tmp/ray/session_2023-12-12_03-18-27_260854_2295938/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2023-12-12_03-18-27_260854_2295938/sockets/raylet', 'MetricsExportPort': 63404, 'NodeName': '10.20.13.236', 'RuntimeEnvAgentPort': 42330, 'alive': True, 'Resources': {'GPU': 2.0, 'CPU': 12.0, 'accelerator_type:TITAN': 1.0, 'memory': 17423568078.0, 'node:10.20.13.236': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 8711784038.0}, 'Labels': {'ray.io/node_id': '1548755e0d56fbbaea367df3cb590eeb6af479d17a2f1c9b48a2e8a4'}}]


In [14]:
@ray.remote(num_gpus=1)
def train_remote(rank, args):
    # Ray will automatically set CUDA_VISIBLE_DEVICES for each task.
    train_in_ray(rank, args)

def distributed_training(world_size=2):
    args = Config()
    num_gpus = ray.available_resources()['GPU']
    args.world_size = min(world_size, num_gpus)
    results = ray.get([train_remote.remote(i, args) for i in range(args.world_size)])
    print(results)

distributed_training()

(train_remote pid=2301260) rank 0 initializing process group
(train_remote pid=2301260) rank 0 done initializing process group
(train_remote pid=2301260) /home/tmb/proj/webdataset/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(train_remote pid=2301260)   warnings.warn(
(train_remote pid=2301260) /home/tmb/proj/webdataset/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
(train_remote pid=2301260)   warnings.warn(msg)
(train_remote pid=2301260) gs://webdataset/fake-imagenet/imagenet-train.json base: gs://webdataset/fake-imagenet name: imagenet-train nfiles: 1282 nbytes: 31242280960 samples: 128200 cache: ./_cache
(train_remote pid